In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!ls -l /content/chest_xray


total 4
drwxr-xr-x 7 root root 4096 Jul  2 15:02 chest_xray


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import numpy as np


In [ ]:
train_dir = '/content/chest_xray_clean/train'
val_dir = '/content/chest_xray_clean/val'
test_dir = '/content/chest_xray_clean/test'


In [ ]:
IMG_SIZE = (150, 150)  # حجم الصورة لتصغير الدخل

train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,
                                   zoom_range=0.15,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.15,
                                   horizontal_flip=True,
                                   fill_mode="nearest")

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=32,
    class_mode='binary')

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=32,
    class_mode='binary')


Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


In [ ]:
model = models.Sequential([
 layers.Input(shape=(*IMG_SIZE, 3)),  # طبقة الإدخال هنا
    layers.Conv2D(32, (3,3), activation='relu'),    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # binary classification
])


In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [ ]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 377s 2s/step - accuracy: 0.7144 - loss: 0.6106 - val_accuracy: 0.6250 - val_loss: 0.7243
Epoch 2/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 384s 2s/step - accuracy: 0.8572 - loss: 0.3211 - val_accuracy: 0.6875 - val_loss: 0.6488
Epoch 3/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 437s 2s/step - accuracy: 0.8832 - loss: 0.2655 - val_accuracy: 0.6250 - val_loss: 0.9510
Epoch 4/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 375s 2s/step - accuracy: 0.9044 - loss: 0.2247 - val_accuracy: 0.7500 - val_loss: 0.7364
Epoch 5/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 382s 2s/step - accuracy: 0.9091 - loss: 0.2136 - val_accuracy: 0.7500 - val_loss: 0.6171
Epoch 6/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 377s 2s/step - accuracy: 0.9242 - loss: 0.1886 - val_accuracy: 0.8125 - val_loss: 0.3802
Epoch 7/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 397s 2s/step - accuracy: 0.9090 - loss: 0.2159 - val_accuracy: 0.8125 - val_loss: 0.5590
Epoch 8/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 426s 3s/step - accuracy: 0.9275 - loss: 0.1753 - val_accu

In [ ]:
# حفظ النموذج بصيغة H5
model.save('chest_xray_model.h5')

# تحويل النموذج إلى TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# حفظ النموذج بصيغة TFLite
with open('chest_xray_model.tflite', 'wb') as f:
    f.write(tflite_model)

print("TFLite model saved!")


Saved artifact at '/tmp/tmp8n4zdysv'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 150, 150, 3), dtype=tf.float32, name='keras_tensor_10')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  138408302250704: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138408302251280: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138408302251856: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138408302252432: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138408302251664: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138408302253200: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138408302252624: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138408302253968: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138408302253392: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138408302254736: TensorSpec(shape=(), dtype=tf.resource, name=None)
TFLite model 

In [ ]:
import zipfile
import os

zip_path = '/content/drive/MyDrive/archive.zip'  # غيّر هذا المسار حسب موقع ملفك في درايف
extract_path = '/content/chest_xray'  # مجلد تودع فيه الملفات المفكوكة

# تأكد من إنشاء مجلد الإخراج
os.makedirs(extract_path, exist_ok=True)

# فك الضغط
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print('✅ Extraction completed!')


✅ Extraction completed!


In [ ]:
import os

for root, dirs, files in os.walk('/content/chest_xray'):
    print(f'Folder: {root}')
    print(f'Contains {len(files)} files\n')


Folder: /content/chest_xray
Contains 0 files

Folder: /content/chest_xray/chest_xray
Contains 0 files

Folder: /content/chest_xray/chest_xray/__MACOSX
Contains 1 files

Folder: /content/chest_xray/chest_xray/__MACOSX/chest_xray
Contains 3 files

Folder: /content/chest_xray/chest_xray/__MACOSX/chest_xray/val
Contains 1 files

Folder: /content/chest_xray/chest_xray/__MACOSX/chest_xray/val/PNEUMONIA
Contains 9 files

Folder: /content/chest_xray/chest_xray/__MACOSX/chest_xray/val/NORMAL
Contains 9 files

Folder: /content/chest_xray/chest_xray/__MACOSX/chest_xray/test
Contains 3 files

Folder: /content/chest_xray/chest_xray/__MACOSX/chest_xray/test/PNEUMONIA
Contains 390 files

Folder: /content/chest_xray/chest_xray/__MACOSX/chest_xray/test/NORMAL
Contains 234 files

Folder: /content/chest_xray/chest_xray/__MACOSX/chest_xray/train
Contains 3 files

Folder: /content/chest_xray/chest_xray/__MACOSX/chest_xray/train/PNEUMONIA
Contains 3876 files

Folder: /content/chest_xray/chest_xray/__MACOSX/

In [ ]:
import shutil
import os

src = '/content/chest_xray/chest_xray/chest_xray'
dst = '/content/chest_xray_clean'

# انسخ فقط المجلدات train, val, test للوجهة الجديدة
shutil.copytree(src, dst)


'/content/chest_xray_clean'